In [91]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.dates as mdates

In [92]:
# Q6 a

df_returns = pd.read_csv('data/preprocess/merged_three_month_interbank_rates.csv')
df_returns.set_index('date', inplace=True)
df_returns.head()

# add carry columns (foreign return - usd return)
df_returns['carry_AUD'] = df_returns['three_month_interbank_rate_AUD'] - df_returns['three_month_interbank_rate_USD']
df_returns['carry_CHF'] = df_returns['three_month_interbank_rate_CHF'] - df_returns['three_month_interbank_rate_USD']
df_returns['carry_EUR'] = df_returns['three_month_interbank_rate_EUR'] - df_returns['three_month_interbank_rate_USD']
df_returns['carry_GBP'] = df_returns['three_month_interbank_rate_GBP'] - df_returns['three_month_interbank_rate_USD']
df_returns['carry_JPY'] = df_returns['three_month_interbank_rate_JPY'] - df_returns['three_month_interbank_rate_USD']


# create a new dataframe with carry returns
df_carry = df_returns[['carry_AUD', 'carry_CHF', 'carry_EUR', 'carry_GBP', 'carry_JPY']].copy()

# Convert the 'date' column to datetime format and shift to the first day of the next month
df_tbills = pd.read_csv('data/preprocess/tbills.csv')
df_tbills.set_index('date', inplace=True)

# To match the hedged return we drop the first row
tbill = df_tbills.rename(columns={"rf": "TBill1Mo"})["TBill1Mo"].iloc[1:]

div_df = pd.read_csv("data/q3/div_return.csv")
div_df.set_index('date', inplace=True)
div = div_df["DIV_return"] 

In [93]:
# extract the rank of each currency's carry return
# for simplicity, we sort by date, drop the date, rank, and add the date back
df_rank = df_carry.rank(axis=1, method="first", ascending=True)

# calculate weights based on shifted ranks w_i = rank_i - (N + 1) / 2
N = 5 # Number of currency         
mean_rank_curr = (N + 1) / 2.0 

# Compute MOM weights
raw_w_curr = (df_rank - mean_rank_curr)
Z = 8.0 / 24.0
w_curr = Z * raw_w_curr

print(df_carry.head())
print(raw_w_curr.head())

#-------------OLD ANSWER-------------#
#carry_cols = [col for col in df_carry.columns if col.startswith('carry_')]

# rank the carry returns
#df_rank = df_carry[carry_cols].rank(axis=1, method='first') 
#df_rank.columns = [col.replace('carry_', 'rank_') for col in df_rank.columns]

# calculate weights based on shifted ranks w_i = rank_i - (N + 1) / 2
#N = 5 # Number of currency

#df_weights = df_rank.sub((N + 1) / 2, axis=0)
#df_weights.columns = [col.replace('rank_', 'weight_') for col in df_weights.columns]

#df_weights.insert(0, 'date', df_carry['date'])

#print(df_carry.head())
#print(df_weights.head())

            carry_AUD  carry_CHF  carry_EUR  carry_GBP  carry_JPY
date                                                             
2002-05-01     0.0302  -0.006565   0.016471   0.023337    -0.0174
2002-06-01     0.0326  -0.006604   0.016540   0.023676    -0.0172
2002-07-01     0.0319  -0.007869   0.016200   0.022705    -0.0170
2002-08-01     0.0323  -0.010617   0.016219   0.022565    -0.0166
2002-09-01     0.0316  -0.011470   0.015501   0.022287    -0.0168
            carry_AUD  carry_CHF  carry_EUR  carry_GBP  carry_JPY
date                                                             
2002-05-01        2.0       -1.0        0.0        1.0       -2.0
2002-06-01        2.0       -1.0        0.0        1.0       -2.0
2002-07-01        2.0       -1.0        0.0        1.0       -2.0
2002-08-01        2.0       -1.0        0.0        1.0       -2.0
2002-09-01        2.0       -1.0        0.0        1.0       -2.0


In [94]:
#-----------OLD ANSWER----------------#
# compute weights using formula: w_i = Z * (rank_i - (N + 1) / 2)

#weights = df_weights.drop(columns='date')
#weights.columns = weights.columns.str.replace('weight_', '', regex=False)

#print("before normalize \n", weights.head())

#def normalize_weights(row):
#    pos_sum = row[row > 0].sum()
#    neg_sum = row[row < 0].sum()
#    Z_pos = 1 / pos_sum if pos_sum != 0 else 0
#    Z_neg = 1 / -neg_sum if neg_sum != 0 else 0
#    return row.apply(lambda x: x * Z_pos if x > 0 else x * Z_neg)

#df_norm_weights = weights.apply(normalize_weights, axis=1)

#print("after normalize \n", df_norm_weights.head())

#df_norm_weights.columns = 'norm_weights_' + df_norm_weights.columns
#df_norm_weights.insert(0, 'date', df_carry['date'])
#df_norm_weights['date'] = pd.to_datetime(df_norm_weights['date'])
#df_norm_weights = df_norm_weights.set_index('date')

#print("finale \n", df_norm_weights.head())
#df_norm_weights.to_csv("data/q6/norm_weights.csv")

print("finale \n", w_curr.head())
w_curr.rename(columns={"carry_AUD":"norm_weights_AUD", 
                       "carry_CHF":"norm_weights_CHF", 
                       "carry_EUR":"norm_weights_EUR", 
                       "carry_GBP":"norm_weights_GBP", 
                       "carry_JPY":"norm_weights_JPY"}).to_csv("data/q6/norm_weights.csv")

finale 
             carry_AUD  carry_CHF  carry_EUR  carry_GBP  carry_JPY
date                                                             
2002-05-01   0.666667  -0.333333        0.0   0.333333  -0.666667
2002-06-01   0.666667  -0.333333        0.0   0.333333  -0.666667
2002-07-01   0.666667  -0.333333        0.0   0.333333  -0.666667
2002-08-01   0.666667  -0.333333        0.0   0.333333  -0.666667
2002-09-01   0.666667  -0.333333        0.0   0.333333  -0.666667


In [95]:
df_excess_return = pd.read_csv("data/q3/excess_returns.csv")
df_excess_return.set_index('date', inplace=True)

# France and Germany are the same
df = df_excess_return.drop(columns = ['X_germany'])

# Drop the first row because X_t+1 and rename for product
w_curr = w_curr.iloc[1:]
dummy_w = w_curr.rename(columns={"carry_AUD":"X_australia", 
                       "carry_CHF":"X_switzerland", 
                       "carry_EUR":"X_france", 
                       "carry_GBP":"X_uk", 
                       "carry_JPY":"X_japan"})

carry_ret = (dummy_w * df).sum(axis=1, skipna=False)

# Decompose the “long leg” (w>0) and “short leg” (w<0) returns:
long_w_carry  = dummy_w.clip(lower=0)
short_w_carry = dummy_w.clip(upper=0)

carry_long_ret  = (long_w_carry  * df).sum(axis=1, skipna=False)
carry_short_ret = (short_w_carry * df).sum(axis=1, skipna=False)

# Align with T-Bill series, drop NaNs (the first 12 months) for clean statistics:
carry_df = pd.DataFrame({
    "CARRY": carry_ret,
    "LONG":  carry_long_ret,
    "SHORT": carry_short_ret,   
    "TBill": tbill
}).dropna()

#------------OLD ANSWER--------------#
#df_hedged_returns = pd.read_csv("data/q3/hedged_returns.csv", parse_dates=['date'], index_col='date')

#print("hedged returns \n", df_hedged_returns.head())
#print("france = ",df_hedged_returns['france_hedged_return_usd'].mean())
#print("germany = ",df_hedged_returns['germany_hedged_return_usd'].mean())
# choosing france for hedging in EUR since it has a better mean hedged return than germany


#df_carry_returns = df_hedged_returns.join(df_norm_weights, how='inner')

#df_carry_returns['australia_carry_return'] = df_carry_returns['australia_hedged_return_usd'] * df_carry_returns['norm_weights_AUD']
#df_carry_returns['switzerland_carry_return'] = df_carry_returns['switzerland_hedged_return_usd'] * df_carry_returns['norm_weights_CHF']
#df_carry_returns['france_carry_return'] = df_carry_returns['france_hedged_return_usd'] * df_carry_returns['norm_weights_EUR']
#df_carry_returns['uk_carry_return'] = df_carry_returns['uk_hedged_return_usd'] * df_carry_returns['norm_weights_GBP']
#df_carry_returns['japan_carry_return'] = df_carry_returns['japan_hedged_return_usd'] * df_carry_returns['norm_weights_JPY']
#df_carry_returns['strategy_carry_return'] = df_carry_returns[['australia_carry_return', 'switzerland_carry_return', 'france_carry_return', 'uk_carry_return', 'japan_carry_return']].sum(axis=1)

#df_carry_returns = df_carry_returns.drop(columns=[col for col in df_carry_returns.columns if 'norm_weights' in col or 'hedged_return' in col])
#df_carry_returns.to_csv("data/q6/carry_returns.csv")

#print("carry returns \n", df_carry_returns.head())

#carry_cols = [col for col in df_carry_returns.columns if col.endswith('_carry_return')]
#total_returns = df_carry_returns[carry_cols].sum()

#print("Total Carry Returns:\n", total_returns)

# TODO plot the returns, and the strategy return, verify if right data (countries vs currency) taken, compare results (seem strange)

In [96]:
# Q6 b
# Helper to compute annualized mean, vol, and Sharpe
def annualized_stats(returns, rf = None, period = 12, zero_cost = False):
    """
    Given two pandas Series of monthly returns (decimal):
      - returns: strategy returns
      - rf:      risk-free returns
    Returns (annualized_mean, annualized_vol, annualized_sharpe).
    """
    mean_annual_return = period * returns.mean()
    std_annual_return = np.sqrt(period) * returns.std()
    if zero_cost:
        return mean_annual_return, std_annual_return, (mean_annual_return/std_annual_return)
    rf_annual = period * rf.mean()
    excess_ret = mean_annual_return - rf_annual
    sharpe_ratio = excess_ret / std_annual_return
    return mean_annual_return, std_annual_return, sharpe_ratio

mean_carry, sigma_carry, sharpe_carry = annualized_stats(carry_df["CARRY"], zero_cost=True)
mean_mL,  sigma_mL,  sharpe_mL  = annualized_stats(carry_df["LONG"], carry_df["TBill"])
mean_mS,  sigma_mS,  sharpe_mS  = annualized_stats(carry_df["SHORT"], carry_df["TBill"])

# One-sample t-test: H0: mean(CARRY) = 0
t_stat_mom, p_val_mom = stats.ttest_1samp(carry_df["CARRY"], popmean=0.0)

print("=== CARRY Strategy Statistics ===")
print(f"Overall CARRY       : Annualized Return = {mean_carry:.2%}, "
      f"Volatility = {sigma_carry:.2%}, Sharpe = {sharpe_carry:.2f}")
print(f"CARRY Long Leg      : Annualized Return = {mean_mL:.2%}, "
      f"Volatility = {sigma_mL:.2%}, Sharpe = {sharpe_mL:.2f}")
print(f"CARRY Short Leg     : Annualized Return = {mean_mS:.2%}, "
      f"Volatility = {sigma_mS:.2%}, Sharpe = {sharpe_mS:.2f}")
print(f"CARRY mean-return t-stat = {t_stat_mom:.3f}, p-value = {p_val_mom:.3f}\n")

alpha = 0.05  # typical significance level
if p_val_mom < alpha:
    print("The strategy's mean return is significantly different from zero.")
else:
    print("The strategy's mean return is not significantly different from zero.")


carry_df['CARRY'].to_csv('data/q6/carry_returns.csv')

#-------------------OLD ANSWER----------------------#

#def return_stats(returns, periods_per_year=12, risk_free_rate=0):
#    mean = returns.mean()
#    std = returns.std()

#    sharpe = ((mean * periods_per_year) - risk_free_rate) / (std * np.sqrt(periods_per_year))

#    return mean, std, sharpe

#print("Long leg = UK, Hhort legs = AUD, CHF, JPY\n")

#uk_stats = return_stats(df_carry_returns['uk_carry_return'])
#print(f"UK Carry Return :\n Mean: {uk_stats[0]}, Std: {uk_stats[1]}, Sharpe Ratio: {uk_stats[2]}")

#australia_stats = return_stats(df_carry_returns['australia_carry_return'])
#print(f"\nAustralia Carry Return :\n Mean: {australia_stats[0]}, Std: {australia_stats[1]}, Sharpe Ratio: {australia_stats[2]}")
#switzerland_stats = return_stats(df_carry_returns['switzerland_carry_return'])
#print(f"Switzerland Carry Return :\n Mean: {switzerland_stats[0]}, Std: {switzerland_stats[1]}, Sharpe Ratio: {switzerland_stats[2]}")
#japan_stats = return_stats(df_carry_returns['japan_carry_return'])
#print(f"Japan Carry Return :\n Mean: {japan_stats[0]}, Std: {japan_stats[1]}, Sharpe Ratio: {japan_stats[2]}")

#print("Full strategy stats:\n")
#strategy_stats = return_stats(df_carry_returns['strategy_carry_return'])
#print(f"Strategy Carry Return :\n Mean: {strategy_stats[0]}, Std: {strategy_stats[1]}, Sharpe Ratio: {strategy_stats[2]}")

# TODO plot long leg vs short legs vs strategy, compare results (seem strange), test if strat is statistically significantly different from long 0

=== CARRY Strategy Statistics ===
Overall CARRY       : Annualized Return = 36.69%, Volatility = 8.91%, Sharpe = 4.12
CARRY Long Leg      : Annualized Return = 16.97%, Volatility = 9.78%, Sharpe = 1.58
CARRY Short Leg     : Annualized Return = 19.73%, Volatility = 9.00%, Sharpe = 2.03
CARRY mean-return t-stat = 19.579, p-value = 0.000

The strategy's mean return is significantly different from zero.


In [97]:
# Q6 c

# Regress CARRY on DIV (Newey-West errors, lag=1)
reg_mom_div = pd.DataFrame({
    "CARRY": carry_ret.iloc[60:],
    "DIV": div
}).dropna()

X_m = sm.add_constant(reg_mom_div["DIV"])
y_m = reg_mom_div["CARRY"]
carry_on_div = sm.OLS(y_m, X_m).fit()

print(">>> Regression of CARRY on DIV:")
print(carry_on_div.summary())
print("\n")

>>> Regression of CARRY on DIV:
                            OLS Regression Results                            
Dep. Variable:                  CARRY   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.964
Date:              mar., 10 juin 2025   Prob (F-statistic):              0.163
Time:                        16:49:53   Log-Likelihood:                 484.33
No. Observations:                 211   AIC:                            -964.7
Df Residuals:                     209   BIC:                            -958.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.023